# 上級編1

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
# 生成多項式から生成行列を作る場合
# g = np.array([0, 0, 0, 1, 1, 0, 1])
# g = np.array([0, 0, 1, 1, 1, 0, 1])

# 巡回問7
g = np.array([0, 0, 1, 0, 1, 1, 1])

k = np.argmax(g) + 1
n = g.shape[0]
G = np.stack([g] * k, 0)
for i, row in enumerate(G):
    if i >= 1:
        G[i] = np.roll(G[i - 1], -1)
G = np.matrix(G, dtype="u1")
print(G)

[[0 0 1 0 1 1 1]
 [0 1 0 1 1 1 0]
 [1 0 1 1 1 0 0]]


In [3]:
# ブロック符号問1の生成行列
G = np.matrix([
    [1, 0, 1, 0, 1, 1],
    [1, 1, 1, 1, 0, 1],
    [0, 1, 1, 0, 0, 1]
], dtype="u1")

# 崩したハミング符号
# 1ビット以下の誤りのみ検出可能
# 完全符号
# G = np.matrix([
#     [1, 1, 0, 0, 1, 0, 0],
#     [1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

# G = np.matrix([
#     [0, 1, 1, 1, 1, 0, 0, 0],
#     [1, 1, 1, 0, 0, 1, 0, 0],
#     [1, 1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

k, n = G.shape
print(G)
k, n

[[1 0 1 0 1 1]
 [1 1 1 1 0 1]
 [0 1 1 0 0 1]]


(3, 6)

## 掃き出し法で組織符号の生成行列に変換

In [5]:
k = G.shape[0]

A = np.matrix(np.identity(k), dtype="u1")
G_src = G.copy()[:,:k]
I_k = np.identity(k)
# print(G_src)
for i in range(k):
#     print(f"掃き出し{i + 1}回目")
    col_i = np.array(G_src)[:,i]
    # 対角成分が0
    # 他の行と入れ替え
    if col_i[i] == 0:
        # 入れ替え対象の行番号
        j = np.where(col_i == 1)[0][-1]
        A_i = I_k.copy()
        A_i[i] = I_k[j].copy()
        A_i[j] = I_k[i].copy()
        A_i = np.matrix(A_i, dtype="u1")
        A = A_i * A % 2
        G_src = A_i * G_src % 2
#         print(G_src)
        col_i = np.array(G_src)[:,i]
    A_i =I_k.copy()
    A_i[:,i] = col_i
    A_i = np.matrix(A_i, dtype="u1")
    A = A_i * A % 2
    # print(f"A_i=\n{A_i}")
    # print(f"A=\n{A}")
    G_src = A_i * G_src % 2
#     print(G_src)

Tissue_G = A * G % 2
print(f"A=\n{A}")
print(f"G=\n{Tissue_G}")

A=
[[0 1 1]
 [1 1 0]
 [1 1 1]]
G=
[[1 0 0 1 0 0]
 [0 1 0 1 1 0]
 [0 0 1 1 1 1]]


## 標準アレイ

In [8]:
# 非零の全符号生成
all_src_ex0 = np.matrix([[int(j) for j in format(i, f"0{k}b")] for i in range(1, 2 ** k)], dtype="u1")
all_code_ex0 = all_src_ex0 * Tissue_G % 2

# 巡回問7. 手書き
# all_code_ex0_str = ["0010111", "0101110", "1011100", "0111001", "1110010", "1100101", "1001011"]
# all_code_ex0 = np.matrix([[int(j) for j in moji] for moji in all_code_ex0_str])

all_code_ex0_str = ["".join(str(i) for i in l) for l in all_code_ex0.tolist()]
all_code_ex0_str

['001111', '010110', '011001', '100100', '101011', '110010', '111101']

In [35]:
# 1ビットエラー全パターン?
all_error_str = [format(2 ** i, f"0{n}b") for i in range(n)]
all_one_bit_error = [format(2 ** i, f"0{n}b") for i in range(n)]
custom_error_str = []
# print(all_one_bit_error)
# 適当に追加
# all_error_str += ["000011"]
# 課題の (8, 4) 符号用
# custom_error_str = ["00000011", "00000101", "00001001", "00010001", "00100001", "01000001", "10000001"]

# 巡回問7, 2bitバースト
# custom_error_str = ["0000011", "0000110", "0001100", "0011000", "0110000", "1100000", "1000001", "1111111"]

all_error_str = all_one_bit_error + custom_error_str

# ブロック符号第2問
# 起こりやすい誤りを勝手に定義
# all_error_str = ["000001", "000010", "000100", "001000", "010000", "000011", "000101"]
all_error_str = ["000001", "000010", "001000", "010000", "100000", "101000", "100001"]

all_error = np.matrix([[int(j) for j in moji] for moji in all_error_str])
all_error_str

['000001', '000010', '001000', '010000', '100000', '101000', '100001']

In [36]:
std_arr_matrix = [[np.matrix((e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
# std_arr_values = [["".join(str(k) for k in (e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
std_arr_values = [["".join(str(k) for k in j.tolist()[0]) for j in i] for i in std_arr_matrix]
# np.array(std_arr_values)
std_arr_df = pd.DataFrame(std_arr_values, columns=all_code_ex0_str, index=all_error_str)
std_arr_df

,001111,010110,011001,100100,101011,110010,111101
000001,001110,010111,011000,100101,101010,110011,111100
000010,001101,010100,011011,100110,101001,110000,111111
001000,000111,011110,010001,101100,100011,111010,110101
010000,011111,000110,001001,110100,111011,100010,101101
100000,101111,110110,111001,000100,001011,010010,011101
101000,100111,111110,110001,001100,000011,011010,010101
100001,101110,110111,111000,000101,001010,010011,011100


In [37]:
# 重複がないか確認
table_set = {"000000"}
table_set |= set(std_arr_df.values.ravel())
table_set |= set(all_code_ex0_str)
table_set |= set(all_error_str)
print(len(table_set))

64


## 各受信後のシンドローム計算

In [9]:
# パリティ生成部分
P = Tissue_G[:, k:]
# print(P)
# 検査行列
H = np.concatenate([P.T, np.identity(n - k, dtype=np.uint8)], 1)
H

matrix([[1, 1, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 0, 1]], dtype=uint8)

In [10]:
syndrome_values = [["".join(str(k) for k in (j * H.T % 2).tolist()[0]) for j in i] for i in std_arr_matrix]
syndrome_df = pd.DataFrame(syndrome_values, columns=all_code_ex0_str, index=all_error_str)
syndrome_df

,0010111,0101110,1011100,0111001,1110010,1100101,1001011
0000001,0001,0001,0001,0001,0001,0001,0001
0000010,0010,0010,0010,0010,0010,0010,0010
0000100,0100,0100,0100,0100,0100,0100,0100
0001000,1000,1000,1000,1000,1000,1000,1000
0010000,0111,0111,0111,0111,0111,0111,0111
0100000,1110,1110,1110,1110,1110,1110,1110
1000000,1011,1011,1011,1011,1011,1011,1011
0000011,0011,0011,0011,0011,0011,0011,0011
0000110,0110,0110,0110,0110,0110,0110,0110
0001100,1100,1100,1100,1100,1100,1100,1100


## 最小ハミング距離

In [11]:
G = np.matrix([
    [1, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 0, 1, 1]
])

k, n = G.shape
print(G)
print(k, n)

all_src_ex0 = [np.matrix([int(j) for j in format(i, f"0{k}b")], dtype=np.uint8) for i in range(1, 1 << k)]
all_code_ex0 = [i * G % 2 for i in all_src_ex0]
code_df = pd.DataFrame(
    ["".join([str(j) for j in i.tolist()[0]]) for i in all_code_ex0],
    index=["".join([str(j) for j in i.tolist()[0]]) for i in all_src_ex0]
)
all_code_ex0_hamming_dist = [i.sum() for i in all_code_ex0]
hamming_dist_min = min(all_code_ex0_hamming_dist)
print(f"最小ハミング距離: {hamming_dist_min}")
# all_code_ex0
code_df

[[1 0 0 0 0 1 1 1]
 [0 1 0 0 1 1 1 0]
 [0 0 1 0 1 1 0 1]
 [0 0 0 1 1 0 1 1]]
4 8
最小ハミング距離: 4


,0
0001,00011011
0010,00101101
0011,00110110
0100,01001110
0101,01010101
0110,01100011
0111,01111000
1000,10000111
1001,10011100
1010,10101010


In [12]:
A_i_dic = Counter(all_code_ex0_hamming_dist)
print(A_i_dic)
p = 0.01
P_u_E = 0
for i, A_i in A_i_dic.items():
    P_u_E += A_i * p ** i * (1 - p) ** (n - i)
P_u_E

Counter({4: 14, 8: 1})


1.344834415e-07